## Note

We were unable to install the `Synth` package locally. Hence, a copy of this notebook along with the output can be seem at the following Minerva Forum [link](https://sle-collaboration.minervaproject.com/?minervaNotebookId=clgq71w0300g70j2idt16bgec&userId=10865&name=Viktor+Tsvil&avatar=https%3A//s3.us-east-1.amazonaws.com/picasso.fixtures/Viktor_Tsvil_10865_2021-01-19T19%3A56%3A07.399Z&readOnly=0&isInstructor=0&signature=2e4dcfe11dd1ed06b9be8f2d910e4991b386b01ca96f99a724c442c29408d1e8).

## Section 2: Replication

In [1]:
# importing the dataset
countries <- read.csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRLoQG5MffxU2wwyrZK4c8STty86OVFyf9VToHBP_3KR1B_BQPUYXtogye9h2IapxIqt0f4P4BZQcSW/pub?gid=1300915934&single=true&output=csv')

# Convert to character to be compatible with dataprep
countries$Country <- as.character(countries$Country) 
head(countries)

,Id,Country,Year,GDP_2011,GFCF,Inflation_prices,Polity,Export_Russia,Import_Russia,GDP_current_US,HDI,Total.Export,Total.Import
,<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,Armenia,1995,2173.36,16.15,175.95,3,NA,NA,1468317350,0.61,351337833,913101627
2,1,Armenia,1996,2336.52,17.88,18.68,-6,108285.75,85594.05,1596968913,0.61,371135639,894135832
3,1,Armenia,1997,2441.18,16.20,13.96,-6,95669.00,59675.00,1639492424,0.62,332378720,955405626
4,1,Armenia,1998,2639.95,16.22,8.67,5,128305.27,39245.28,1893726437,0.64,359865213,1000634291
5,1,Armenia,1999,2744.43,16.42,0.65,5,56878.06,27511.09,1845482181,0.64,383089747,919091317
6,1,Armenia,2000,2924.75,18.44,-0.79,5,27496.53,43988.44,1911563665,0.65,423629630,957156467


In [5]:
install.packages('synth')
require(Synth)

# obtain the score of trade dependency with Russia
countries$export_russia_share_GDP = countries$Import_Russia * 1000 / countries$GDP_current_US 
countries$import_russia_share_GDP = countries$Export_Russia * 1000 / countries$GDP_current_US 
countries$export_russia_share_export = countries$export_russia_share_GDP * 100 / countries$Total.Export	
countries$import_russia_share_import = countries$import_russia_share_GDP * 100 / countries$Total.Import	
countries$russia_trade_dependence = countries$export_russia_share_GDP + countries$import_russia_share_GDP # aggregate trade amount  

syn <- dataprep(
  foo = countries, 
  predictors = c('russia_trade_dependence', 'HDI', 'Polity', 'Inflation_prices', 'GFCF'), 
  special.predictors = list(
    list('GDP_2011', 2000, "mean"),
    list('GDP_2011', 2012, "mean")
  ),
  dependent = "GDP_2011", 
  unit.variable = "Id",
  time.variable = "Year",
  controls.identifier = c(1:17),  # control units
  treatment.identifier = 18,  # treatment unit
  time.predictors.prior = c(1995:2012),  # pre-treatment
  time.optimize.ssr = c(1995:2012), 
  time.plot = c(1995:2017),  # total timeframe
  unit.names.variable = 'Country')

# perform optimization 
synth.out <- synth(syn)

# show the results 
synth.tables <- synth.tab(
  dataprep.res = syn,
  synth.res = synth.out)
print(synth.tables)

# show the differences in GDP between Ukraine and synthetic unit
syn$Y1plot - (syn$Y0plot %*% synth.out$solution.w)
mean((syn$Y1plot - (syn$Y0plot %*% synth.out$solution.w))[2013:2017])

path.plot(dataprep.res = syn, synth.res = synth.out, Legend.position=c("topleft"), Ylab = 'per capita GDP, 2011 international dollars (PPP)')
abline(v=2012)

Installing package into ‘/home/viktor/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is unspecified)

Warning message:
“package ‘synth’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Loading required package: Synth

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘Synth’”


ERROR: Error in dataprep(foo = countries, predictors = c("russia_trade_dependence", : could not find function "dataprep"


## Part 3 Extension 1 (Extending Timeframe)

In [ ]:
# extended data link
p3i1.link = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRLoQG5MffxU2wwyrZK4c8STty86OVFyf9VToHBP_3KR1B_BQPUYXtogye9h2IapxIqt0f4P4BZQcSW/pub?gid=1793425122&single=true&output=csv"
countries <- read.csv(p3i1.link)

countries$Country <- as.character(countries$Country)
countries$export_russia_share_GDP = countries$Import_Russia * 1000 / countries$GDP_current_US # in thouands of USD 
countries$import_russia_share_GDP = countries$Export_Russia * 1000 / countries$GDP_current_US # in thousands of USD
countries$export_russia_share_export = countries$export_russia_share_GDP * 100 / countries$Total.Export	
countries$import_russia_share_import = countries$import_russia_share_GDP * 100 / countries$Total.Import	
countries$russia_trade_dependence = countries$export_russia_share_GDP + countries$import_russia_share_GDP # aggregate trade amount  

syn <- dataprep(
  foo = countries, 
  predictors = c('russia_trade_dependence', 'HDI', 'Polity', 'Inflation_prices', 'GFCF'), 
  special.predictors = list(
    list('GDPpc2017', 2000, "mean"),
    list('GDPpc2017', 2012, "mean"), 
    list('GDPpc2017', 2008, "mean") 
  ),
  dependent = "GDPpc2017", 
  unit.variable = "Id",
  time.variable = "Time", 
  controls.identifier = c(1:17), 
  treatment.identifier = 18, 
  time.predictors.prior = c(1995:2012),
  time.optimize.ssr = c(1995:2012), 
  time.plot = c(1995:2021),
  unit.names.variable = 'Country')

#Perform the optimization procedure
synth.out <- synth(
    syn,
    custom.v = c(0.2653768, 0.0001054143, 0.02362027, 0.031404951, 0.005282442, 0.07510658, 0.2386713, 0.3604323), #uncomment
)

#Show the results of SCM
synth.tables <- synth.tab(
  dataprep.res = syn,
  synth.res = synth.out)
print(synth.tables)

#Differences in GDP between Ukraine and synthetic unit
syn$Y1plot - (syn$Y0plot %*% synth.out$solution.w)

path.plot(dataprep.res = syn, synth.res = synth.out, Legend.position=c("topleft"), Ylab = 'per capita GDP, 2017 international dollars (PPP)', Xlab = 'Time\nOriginal Donor Pool')
abline(v=2012)
abline(v=2012, type='dashed')

## Part 3 Extension Idea 1 + 2 (Extending Timeframe + Excluding Countries)


In [ ]:
head(countries)

syn <- dataprep(
  foo = countries, 
  predictors = c('russia_trade_dependence', 'HDI', 'Polity', 'Inflation_prices', 'GFCF'), 
  special.predictors = list(
    list('GDPpc2017', 2000, "mean"),
    list('GDPpc2017', 2012, "mean")
  ),
  dependent = "GDPpc2017", 
  unit.variable = "Id",
  time.variable = "Time",
  controls.identifier = c(3:7, 9:14, 17),      # <--------- look at this line excludes 5 countries with the 6+ month long conflicts in time between 1995 and 2012 ('Armenia', 'Azerbaijan', 'Tajikistan', 'Kazakhstan', 'Kyrgyz Republic')
  treatment.identifier = 18, 
  time.predictors.prior = c(1995:2012),
  time.optimize.ssr = c(1995:2012), 
  time.plot = c(1995:2021),
  unit.names.variable = 'Country')

#Perform the optimization procedure
synth.out <- synth(
    syn,
)

#Show the results of SCM
synth.tables <- synth.tab(
  dataprep.res = syn,
  synth.res = synth.out)
print(synth.tables)

#Differences in GDP between Ukraine and synthetic unit
syn$Y1plot - (syn$Y0plot %*% synth.out$solution.w)

path.plot(dataprep.res = syn, synth.res = synth.out, Legend.position=c("topleft"), Ylab = 'per capita GDP, 2017 international dollars (PPP)', Xlab = 'Time\nAdjusted Donor Pool')
abline(v=2012)